## Exercise: Pricing a European Call Option under Risk Neutrality

#### John Stachurski

Let's price a European option under the assumption of risk neutrality (for simplicity).

Suppose that the current time is $t=0$ and the expiry date is $n$.

We need to evaluate

$$ P_0 = \beta^n \mathbb E_0 \max\{ S_n - K, 0 \} $$

given 

* the discount factor $\beta$
* the strike price $K$
* the stochastic process $\{S_t\}$



A common model for $\{S_t\}$ is

$$ \ln \frac{S_{t+1}}{S_t} = \mu + \sigma \xi_{t+1} $$

where $\{ \xi_t \}$ is IID and standard normal.  However, its predictions are in some ways counterfactual.  For example, volatility is not stationary but rather changes over time.  Here's an improved version:

$$ \ln \frac{S_{t+1}}{S_t} = \mu + \sigma_t \xi_{t+1} $$

where 

$$ 
    \sigma_t = \exp(h_t), 
    \quad
        h_{t+1} = \rho h_t + \nu \eta_{t+1}
$$


Compute the price of the option $P_0$ by Monte Carlo, averaging over realizations $S_n^1, \ldots, S_n^M$ of $S_n$ and appealing to the law of large numbers:

$$ \mathbb E_0 \max\{ S_n - K, 0 \} 
    \approx
    \frac{1}{M} \sum_{m=1}^M \max \{S_n^m - K, 0 \}
    $$

Use the following parameters:

In [20]:
β = 0.96
μ = 0.005
S0 = 10
h0 = 0
K = 100
n = 10
ρ = 0.5
ν = 0.01
M = 5_000_000

**Suggestion**: Start without jitting your functions, as jitted functions are harder to debug.  Chose a smaller value for `M` and try to get your code to run.  Then think about jitting.

The distribution of prices is heavy tailed, so the result has high variance even for large `M`.  My best estimate is around $1,530.

### Solution

In [1]:
import numpy as np
from numpy.random import randn
from numba import jit, prange
from quantecon import tic, toc

Here's a solution that's jitted but not parallelized.  A parallelized solution is below.

In [2]:
@jit(nopython=True)
def compute_call_price(β=0.96,
                       μ=0.005,
                       S0=10,
                       h0=0,
                       K=100,
                       n=10,
                       ρ=0.5,
                       ν=0.01,
                       M=5_000_000):
    current_sum = 0.0
    for m in range(M):
        s = np.log(S0)
        h = h0
        for t in range(n):
            s = s + μ + np.exp(h) * randn()
            h = ρ * h + ν * randn()
        current_sum += np.maximum(np.exp(s) - K, 0)
        
    return β**n + current_sum / M

In [23]:
tic()
price = compute_call_price()
toc()

TOC: Elapsed: 0:00:5.79


5.798811912536621

In [24]:
tic()
price = compute_call_price()
toc()

TOC: Elapsed: 0:00:5.73


5.730069875717163

In [25]:
price

1479.8757985970683

Let's try to parallelize this task.

In [8]:
@jit(nopython=True, parallel=True)
def compute_call_price_parallel(β=0.96,
                                   μ=0.005,
                                   S0=10,
                                   h0=0,
                                   K=100,
                                   n=10,
                                   ρ=0.5,
                                   ν=0.01,
                                   M=50_000_000):
    current_sum = 0.0
    for m in prange(M):
        s = np.log(S0)
        h = h0
        for t in range(n):
            s = s + μ + np.exp(h) * randn()
            h = ρ * h + ν * randn()
        current_sum += np.maximum(np.exp(s) - K, 0)
        
    return β**n + current_sum / M

In [9]:
tic()
price = compute_call_price_parallel()
toc()

TOC: Elapsed: 0:00:18.69


18.690572500228882

In [6]:
tic()
price = compute_call_price_parallel()
toc()

TOC: Elapsed: 0:00:5.51


5.510042190551758

In [10]:
price

1526.6196097772884